In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import plotly.plotly as plotly
import plotly.graph_objs as go

### This notebook uses a dataset from Kaggle (https://www.kaggle.com/jameslko/gun-violence-data) that contains gun violence data in America from 2013-2018. The entries for 2013 are incomplete, and only data up to March of 2018 are included. Not all gun incidents, including the Las Vegas Shooting, are included. Because that specific incident is  so important, it was added into the dataset. 

#### Downloading the data and adding Vegas statistics to the dataset

In [ ]:
data = pd.read_csv('./gun-violence-data.csv')

In [ ]:
lasvegas = [999999999, '2017-10-01', 'Nevada', 'Las Vegas', None, 59, 411, None, None, None, None, 
            None, None, None, 36.095, None, -115.171667, 47, None, None, None, None, None, None, 
            None, None, None, None, None]
data.loc[len(data)] = lasvegas

In [ ]:
data.tail()

#### Add columns that break up the date by year and month, and calculate total number of victims per gun incident. The total number of victims is defined as the sum of the number of people killed and the number of people injured.

In [ ]:
data['year'] = data['date'].apply(lambda x:pd.to_datetime(x).year)
data['month'] = data['date'].apply(lambda x:pd.to_datetime(x).month)
data['victims'] = data['n_killed'] + data['n_injured']

In [ ]:
data.head()

#### Read in population data. Data was downloaded from the Center for Disease Control website (https://wonder.cdc.gov/population-projections.html)

In [ ]:
pop_data = pd.read_table('./state_pop_projections_2014-2018.txt', dtype = {'Year': np.int32, 'Projected Populations': np.int32})

In [ ]:
pop_data.head()

#### Create a dataframe to properly display the population data

In [ ]:
state_dict = {'Alabama': "AL", 'Alaska': "AK", "Arizona": "AZ", "Arkansas": "AR",
              "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
              "District of Columbia": "DC", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI",
              "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
              "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME",
              "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
              "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE",
              "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM",
              "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH",
              "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI",
              "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX",
              "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
              "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"}

years = (2014,2015,2016,2017,2018)

In [ ]:
pop_data_list = []

for i in state_dict:
    state_pop = pop_data.loc[pop_data["State"] == i]["Projected Populations"].tolist()
    pop_data_list.append({"State" : i, "2014" : state_pop[0],
                         "2015" : state_pop[1], "2016" : state_pop[2], "2017" : state_pop[3],
                         "2018" : state_pop[4]} )
    
pop_data_cleaned = pd.DataFrame.from_dict(pop_data_list)
pop_data_cleaned = pop_data_cleaned[["State", "2014", "2015", "2016", "2017", "2018"]]

In [ ]:
pop_data_cleaned.head()

#### Isolate the victim counts into its own dataframe, ordered by state and year.

In [ ]:
victim_counts = []

for state in state_dict:
    state_data = []
    temp = data.loc[data['state'] == state]
    for year in years:
        state_year = temp.loc[temp['year'] == year]
        state_data.append(state_year['victims'].sum())
    victim_counts.append({'State' : state, "2014" : state_data[0],
                         "2015" : state_data[1], "2016" : state_data[2], "2017" : state_data[3],
                         "2018" : state_data[4]})
    
victim_data_cleaned = pd.DataFrame.from_dict(victim_counts)
victim_data_cleaned = victim_data_cleaned[["State", "2014", "2015", "2016", "2017", "2018"]]

In [ ]:
victim_data_cleaned.head()

#### Reformat the state name to its respective two letter abbreviation so Plotly can read it properly

In [ ]:
victim_data_cleaned['State'].replace(state_dict, inplace = True)

#### Create another dataset for victim counts per 100K people in that state. 

In [ ]:
per_pop = []

for i in range(51):
    per_pop.append({"State": pop_data_cleaned.at[i,"State"]})
    for year in years:
        per_pop[i][str(year)] = (victim_data_cleaned.at[i,str(year)])/(pop_data_cleaned.at[i,str(year)])*100000
        
per_pop_data_cleaned = pd.DataFrame.from_dict(per_pop)
per_pop_data_cleaned = per_pop_data_cleaned[["State", "2014", "2015", "2016", "2017", "2018"]]

In [ ]:
per_pop_data_cleaned.head()

In [ ]:
per_pop_data_cleaned['State'].replace(state_dict, inplace = True)

In [ ]:
no_dc = per_pop_data_cleaned[per_pop_data_cleaned.State != 'DC']

In [ ]:
scl = [[0.0, 'rgb(255,255,255)'], [0.1, 'rgb(255,230,230)'], [0.2, 'rgb(255,205,205)'],\
       [0.3, 'rgb(255,180,180)'], [0.4, '255,155,155)'], [0.5, 'rgb(255,130,130)'],\
       [0.6, 'rgb(255,105,105)'],[0.8, 'rgb(255,80,80)'], [0.9, 'rgb(255,40,40)'],\
       [1.0, 'rgb(255,0,0)']]

#### Plot the total victim count per 100K people using Plotly's choropleth map feature. Slider bars are included to scroll between year (2014-2018). 

In [ ]:
all_year_map_data = []

for year in years:
    map_data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = True,
        locations = no_dc['State'],
        z = no_dc[str(year)].astype(float),
        name = year,
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        visible = False,
        colorbar = dict(
            title = "Victims (Deaths + Injuries)")
        ) ]
    all_year_map_data.extend(map_data)
    
all_year_map_data[0]['visible'] = True

steps = []
yr = 0
for i in range(0,len(all_year_map_data)):
    step = dict(method = "restyle",
                args = ["visible", [False]*len(all_year_map_data)],
                label = years[yr]) 
    step['args'][1][i] = True
    steps.append(step)
    yr += 1

sliders = [dict(active = 10,
                currentvalue = {"prefix": "Year: "},
                pad = {"t": 50},
                steps = steps)]

layout = dict(
        title = 'US Gun Victims by State (per 100k people)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
        sliders = sliders)
    
fig = dict( data=all_year_map_data, layout=layout )
plotly.iplot( fig, filename='gun-rate-map' )

#### Plot the raw total victim counts on another map

In [ ]:
all_year_map_data = []

for year in years:
    map_data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = True,
        locations = victim_data_cleaned['State'],
        z = victim_data_cleaned[str(year)].astype(float),
        locationmode = 'USA-states',
        name = year,
        marker = dict(
            line = dict(
                color = 'rgb(255,255,255)',
                width = 2) ),
        visible = False,
        colorbar = dict(title = "Victims (Deaths + Injuries)") ) ]
    all_year_map_data.extend(map_data)

all_year_map_data[0]['visible'] = True

steps = []
for i in range(len(years)):#range(0,len(all_year_map_data)):
    step = dict( method = "restyle",
            args = ["visible", [False]*len(years)],
            label = years[i] ) 
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active = 10,
            currentvalue = {"prefix": "Year: "},
            pad = {"t": 50},
            steps = steps)]

layout = dict(
        title = 'US Gun Victims by State (Counts)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = False,
            lakecolor = 'rgb(255, 255, 255)'
        ),
        sliders = sliders)
    
fig = dict( data=all_year_map_data, layout=layout )
plotly.iplot( fig, filename='gun-count-map' )

#### Note: The 2017 Vegas shooting does skew the data a bit for Nevada. There were nearly 500 deaths and injuries combined, which is an anomaly for a gun incident, even in the US.

#### These two maps show why count data has to be displayed as a rate when comparing between events over a different size. In this case, state populations are different, so the data has to be converted to a rate. California has just under 10x the population of Alabama, so it would make sense that California has a lot more gun victims than Alabama. If you look at the rate of gun victims per 100K people per state, Alabama actually comes out to be worse for gun victims. California acerages about 8 gun victims per 100K people, while Alabama roughly averages about 20 victims per 100K people. 

#### The difference in counts and rates is the difference between narratives. If the raw counts were displayed, one would believe that California is a haven for gun violence and Alabama is a lot safer. However, when you look at the rate, you see that, in fact, California is a lot safer than the raw total actually makes it. 

#### This is an example why the offset term is included when dealing with counting data and poisson distributions in statistics. Without the offset term in the poisson distribution, you are bound to reach ill-informed conclusions. 

### Isolate the original dataset to only include incidents where there were more than 10 victims. There is no clear definition for what consitutes a mass shooting, so for this scenario, we will define it as a gun incident with more than 10 victims.

In [ ]:
mass_shootings = data.loc[data['victims'] > 10]

#### Using the mass shooting dataset, plot where each incident occurs using their respective latitude/longitude coordinate. Each bubble is centered around the location, and the size is scaled by how many victims there were. 

In [ ]:
limits = [(10,15),(15,20),(25,30),(30,40),(40,100),(100,500)]
colors = ["rgb(128,128,255)","rgb(255,170,0)","rgb(255,128,128)","rgb(255,110,0)","rgb(0,0,255)","rgb(255,0,0)"]
cities = []
scale = 5

for i in range(len(limits)):
    subset = mass_shootings.loc[mass_shootings['victims'] < limits[i][1]]
    subset = subset.loc[subset['victims'] >= limits[i][0]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = subset['longitude'],
        lat = subset['latitude'],
        text = subset['city_or_county'],
        marker = dict(
            size = subset['victims']*scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{} - {} Victims'.format(limits[i][0],limits[i][1]) )
    cities.append(city)

layout = dict(
        title = '2014-2018 US Gun Incidents With 10+ Victims' ,
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict(type='albers usa'),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        )
    )
fig = dict( data=cities, layout=layout )
plotly.iplot( fig, filename='mass-murder-map')

#### Create another dataset of total victim counts per month/year combination. 2018 data only goes up to March, so that is where the dataset ends.

In [ ]:
month_year_dict = []

for year in years:
    year_set = data.loc[data['year'] == year]
    if year == 2018:
        for month in range (1,4):
            month_year_set = year_set.loc[year_set['month'] == month]
            month_year_dict.append({'date': '{:02d}/{}'.format(month, year),
                                    'month': month,
                                    'year': year,
                                    'total victims' : month_year_set['victims'].sum()})
    else:
        for month in range (1,13):
            month_year_set = year_set.loc[year_set['month'] == month]
            month_year_dict.append({'date': '{:02d}/{}'.format(month, year),
                                    'month': month,
                                    'year': year,
                                    'total victims' : month_year_set['victims'].sum()})
                                    
month_year_totals = pd.DataFrame.from_dict(month_year_dict)

## Plotting the number of total victims per month from January 2014 to March 2018

In [ ]:
trace = [go.Scatter(
        x= month_year_totals['date'],
        y= month_year_totals['total victims']
)]
plotly.iplot(trace, filename='gun-victim-month-line')

#### This is a time series with seasonality. In one calendar year, February has the least number of total victims. This could be due to the fact that February has 2-3 less days than other months, but given that the number of victims is substantially less than January and March, there are other factors that are better in explaining this. Conversely, July has the highest number of total victims (except for 2017, but that is because the Las Vegas Shooting had 400+ victims, which would skew the data a bit). The high number of gun victims can be attributed to summer, so people tend to be outside more, and July 4, one of the most violent days annually in America.

## Plot the total number of victims per year (2018 data consists of January to March data)

In [ ]:
year_sums = data.groupby('year')['victims'].sum()
year_sums = year_sums.drop([2013])
year_sums

bar_year = [go.Bar(
            x= years,
            y= year_sums
    )]

plotly.iplot(bar_year, filename='gun-victim-year-bar')

#### As seen from this chart, there is an increasing trend in the number of gun violence victims per year. 